# 🎼🤖 ArmonIA - generación de musica con redes neuronales

## Introducción

armonIA es un proyecto que busca generar música utilizando redes neuronales. El objetivo es crear un modelo capaz de componer música original en diferentes estilos de musica clasica, utilizando técnicas de aprendizaje profundo y procesamiento de lenguaje natural. En este proyecto, se utilizará Torch para construir y entrenar el modelo.


### Prologo

Antes de comenzar con el desarrollo del proyecto, es importante entender los datos que se tienen y lo que cada uno representa, a continuación se lista cada uno de los datos de entrada 

Características de entrada 

| Característica | Tipo de Variable | Descripción                                                                |
|----------------|------------------|----------------------------------------------------------------------------|
| Pitch          | Categórica       | Representa la altura de la nota musical (`note.pitch`).                    |
| Step           | Numérica         | Diferencia entre el `start` de la nota actual y el `start` de la anterior. |
| Duration       | Numérica         | Diferencia entre el `end` y el `start` de la nota actual.                  |
| Velocity       | Numérica         | Representa la velocidad de la nota (`note.velocity`).                      |


Como las variables Step y Duration son numéricas, se normalizan para que estén en el rango [0, 1]. La variable Velocity se normaliza para que esté en el rango [0, 127]. La variable Pitch se convierte a una representación numérica utilizando un diccionario de mapeo.

en este caso no es necesario eliminar los datos duplicados, ya que cada nota es única y no se repite en el tiempo. Sin embargo, es importante asegurarse de qe no haya datos nulos o vacíos en el conjunto de datos.

para la validación del modelo es necesario normalizar los datos, ya que el modelo no podrá predecir correctamente si los datos de entrada no están en el mismo rango que los datos de entrenamiento. Por lo tanto, se debe guardar la media y la desviación estándar de cada variable para poder normalizar los datos de entrada durante la validación.

| salida   | tipo de predicción | metrica   |
|----------|--------------------|-----------|
| pitch    | clasificacion      | accuracy  |
| step     | regresion          | mse / mae |
| duration | regresion          | mse / mae |
| velocity | regresion          | mse / mae |


Se realiza un diseño primero de la arquitectura del modelo, para luego proceder a la implementación del mismo. El modelo se basa en una red neuronal LSTM (Long Short-Term Memory) que es capaz de aprender patrones en secuencias de datos. La arquitectura del modelo se puede ver en la siguiente imagen: 

pasos para mejorar

1. verificar validez de los datos
2. usar embedings para la variable Pitch, debido a que es categórica y no numérica.
3. verificar arquitectura del modelo


<div style="display: flex; gap: 32px; justify-content: center; item-align: center; width: 100%;">
    <!-- <img style="width:400px" src="https://www.researchgate.net/publication/385782855/figure/fig1/AS:11431281290276122@1731539282950/LSTM-model-architecture-for-song-generation-Adapted-from-9.ppm" /> -->
    <img src="./lstm.png" style="width:500px" alt="LSTM model architecture for song generation" />
    <div>
        Se utilizará un modelo LSTM para la generación de música debido a su capacidad para manejar secuencias de datos. El modelo tomará como entrada una secuencia de notas y acordes, y generará una nueva secuencia de notas y acordes como salida.
    </div>
</div>

## Fases del proyecto

### 1. Pre procesamiento de datos

Para entrenar un modelo de generación de música, es necesario contar con un conjunto de datos que contenga ejemplos de música en el estilo deseado. En este caso, se utilizará un conjunto de datos de partituras musicales en formato MIDI. Se utilizarán bibliotecas como `pretty-midi` para procesar y analizar los archivos MIDI.

En esta etapa se realizarán las siguientes tareas:

1. Cargar los archivos MIDI y extraer las notas y acordes.
2. Convertir las notas y acordes en una representación numérica que pueda ser utilizada por el modelo.
3. Dividir los datos en secuencias de longitud fija para facilitar el entrenamiento del modelo.
4. Normalizar los datos para mejorar la convergencia del modelo.
5. Dividir los datos en conjuntos de entrenamiento y validación.



### 2. Construcción, entrenamiento y evaluación del modelo

Para la construcción del modelo se utilizará la biblioteca `torch` para crear una red neuronal LSTM. El modelo tomará como entrada una secuencia de notas y acordes, y generará una nueva secuencia de notas y acordes como salida. Se utilizarán capas LSTM para capturar las dependencias temporales en los datos, y se aplicarán técnicas de regularización como Dropout para evitar el sobreajuste.

Lista de pasos a seguir:

1. Definir la arquitectura del modelo LSTM.
2. Definir la función de pérdida y el optimizador.
3. Implementar el bucle de entrenamiento y validación.
4. Guardar el modelo entrenado para su uso posterior.
5. Evaluar el modelo utilizando métricas como la pérdida y la precisión.


### 3. Generación de música

Para generar música, se utilizará el modelo entrenado para predecir la siguiente nota o acorde dado una secuencia de notas y acordes inicial. Se implementará un algoritmo de muestreo para generar nuevas secuencias de notas y acordes, y se utilizarán técnicas de post-procesamiento para convertir las secuencias generadas en archivos MIDI.

Lista de pasos a seguir:

1. Cargar el modelo entrenado.
2. Definir una secuencia inicial de notas y acordes.
3. Utilizar el modelo para predecir la siguiente nota o acorde.
4. Repetir el proceso para generar una secuencia completa de notas y acordes.
5. Convertir la secuencia generada en un archivo MIDI.

---

## 1. Pre procesamiento de datos

### 1.1 Impotar librerias necesarias

In [1]:
# source venv/bin/activate
#  python -m ipykernel install --user --name=myenv --display-name="Python (myenv)"

In [1]:
import os
import numpy as np
import random
from typing import List, Tuple, Dict, Any, Union 

import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader

import pretty_midi

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

import pandas as pd

# 1.2 Crear funciones para cargar y procesar los archivos MIDI

se crean las funciones necesarias para cargar y procesar los archivos MIDI. Se utiliza la biblioteca `pretty_midi` para cargar los archivos MIDI y extraer las notas y acordes. Se definen funciones para convertir las notas y acordes en una representación numérica, dividir los datos en secuencias de longitud fija, normalizar los datos y dividir los datos en conjuntos de entrenamiento y validación.




In [2]:
from dataset import MusicDataset
from pre import Preprocessor
from model3 import MusicLSTM

In [3]:
def filter_unseen_pitch(X, y, valid_pitches):
    X_filtered = []
    y_filtered = []
    for x_seq, y_note in zip(X, y):
        if y_note[0] in valid_pitches and all(note[0] in valid_pitches for note in x_seq):
            X_filtered.append(x_seq)
            y_filtered.append(y_note)
    return np.array(X_filtered), np.array(y_filtered)


In [4]:
preprocessor = Preprocessor()

# listar todos los archivos midi 
label_encoder = LabelEncoder()

# cargar los archivos midi
_path = "dataset/music_artist/mozart"
midifiles = [os.path.join(_path, f) for f in os.listdir(_path) if f.endswith('.mid')]
random.shuffle(midifiles)

# dividir en train test split 
spit_idx = int(len(midifiles) * 0.8)
train_files = midifiles[:spit_idx]
test_files = midifiles[spit_idx:]
print(f"total: {len(midifiles)}, train: {len(train_files)}, test: {len(test_files)}")


# cargar las notas y extraer las caracteristicas (pitch, step, duration, velocity)
data_train = []
data_test = []

for path in train_files:
    notes = preprocessor.load_midi_files(path)
    features = preprocessor.extract_features(notes)

    data_train.append(features)

for path in test_files:
    notes = preprocessor.load_midi_files(path)
    features = preprocessor.extract_features(notes)

    data_test.append(features)

data_train = np.concatenate(data_train, axis=0)
data_test = np.concatenate(data_test, axis=0)

# convertir a dataframe
df_data_train = preprocessor.get_notes_dataframe(data_train)
df_data_test = preprocessor.get_notes_dataframe(data_test)

df_data_train["pitch"] = df_data_train["pitch"].astype(int)
df_data_test["pitch"] = df_data_test["pitch"].astype(int)

total: 21, train: 16, test: 5


In [5]:
df_data_train.sample(4)

,pitch,step,duration,velocity
12661,73,0.107914,0.053192,47
25214,78,0.212841,0.212842,58
8559,75,0.316589,0.256381,51
16077,70,0.100854,0.104609,70


como se observa en la tabla anterior, se tienen 38524 notas, de las cuales 46 son unicas

In [6]:
def validar_datos_notas(df, nombre="DataFrame"):
    errores = {}

    # Step < 0
    pasos_negativos = df[df["step"] < 0]
    if not pasos_negativos.empty:
        errores["step"] = pasos_negativos
        print(f"⚠️ {nombre}: Hay {len(pasos_negativos)} valores de `step` negativos.")

    # Duration < 0
    duraciones_negativas = df[df["duration"] < 0]
    if not duraciones_negativas.empty:
        errores["duration"] = duraciones_negativas
        print(f"⚠️ {nombre}: Hay {len(duraciones_negativas)} valores de `duration` negativos.")

    # Valores nulos
    nulos = df[df.isnull().any(axis=1)]
    if not nulos.empty:
        errores["nulls"] = nulos
        print(f"⚠️ {nombre}: Hay {len(nulos)} filas con valores nulos.")

    # Valores fuera de rango pitch
    pitch_out_of_range = df[(df["pitch"].astype(int) < 0) | (df["pitch"].astype(int) > 127)]
    if not pitch_out_of_range.empty:
        print(f"⚠️ {len(pitch_out_of_range)} notas tienen pitch fuera del rango MIDI (0–127).")

    # Valores fuera de rango velocity
    velocity_out_of_range = df[(df["velocity"].astype(int) < 0) | (df["velocity"].astype(int) > 127)]
    if not velocity_out_of_range.empty:
        print(f"⚠️ {len(velocity_out_of_range)} notas tienen velocity fuera del rango MIDI (0–127).")

    if not errores:
        print(f"✅ {nombre}: Todo en orden. No se encontraron valores inválidos.")
    
    return errores

validar_datos_notas(df_data_train, "Train")
validar_datos_notas(df_data_test, "Test")


✅ Train: Todo en orden. No se encontraron valores inválidos.
✅ Test: Todo en orden. No se encontraron valores inválidos.


{}

generar secuencias


In [7]:
context_window = 32

# crear secuencias
X_train, y_train = preprocessor.create_sequences(data_train, context_length=context_window)
X_test, y_test = preprocessor.create_sequences(data_test, context_length=context_window)

# normalizar los datos
X_train, y_train = preprocessor.normalize_features(X_train, y_train)
X_test, y_test = preprocessor.normalize_features(X_test, y_test)

# generar el label encoder para las notas
pitches_train = X_train[:, :, 0].flatten().tolist() + y_train[:, 0].tolist()
label_encoder.fit(pitches_train)
valid_pitches = set(map(int, label_encoder.classes_))

# filtrar las secuencias que no contienen notas válidas
X_test, y_test = filter_unseen_pitch(X_test, y_test, valid_pitches)

In [8]:
print(valid_pitches)
X_test

{48, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 91}


array([[[74, 0.0, 0.12249235449911183, 0.7833333333333332],
        [69, 0.0, 0.12249235449911183, 0.5666666666666667],
        [79, 0.24952665423285644, 0.02414488039841352,
         0.6666666666666667],
        ...,
        [81, 0.029924889906557284, 0.024144880398413487,
         0.6499999999999999],
        [79, 0.029924889906557024, 0.02442796785664027,
         0.4833333333333333],
        [78, 0.029924889906557024, 0.027258842438909487,
         0.4833333333333333]],

       [[69, 0.0, 0.12249235449911183, 0.5666666666666667],
        [79, 0.24952665423285644, 0.02414488039841352,
         0.6666666666666667],
        [78, 0.02992488990655712, 0.02414488039841352,
         0.4833333333333333],
        ...,
        [79, 0.029924889906557024, 0.02442796785664027,
         0.4833333333333333],
        [78, 0.029924889906557024, 0.027258842438909487,
         0.4833333333333333],
        [79, 0.03256862186485772, 0.026954225586182757,
         0.4833333333333333]],

       [[79, 0.2

In [9]:
# dataloader
train_dataset = MusicDataset(X_train, y_train, label_encoder)
test_dataset = MusicDataset(X_test, y_test, label_encoder)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

### 2. Construcción del modelo


Características de entrada 

- Pitch (altura de la nota): variable categ´orica que representa la nota musical.
- Step (tiempo): variable numerica que se debe calcular como el start de la nota actual menos el start de la nota anterior, y representa el espacio de tiempo que hay entre la   ota anterior y la nota actual.
- Duration (duración): variable numérica, se calcula como la diferencia entre el tiempo en el que termina la nota actual (end) menos el tiempo en el que inicio la nota (start), o que refleja la duracion en tiempo de la nota.
- velocity (velocidad): variable numérica que representa la velocidad de la nota


Para construir el modelo de generación de música, se utilizará una red neuronal recurrente (RNN) con capas LSTM. Las RNN son adecuadas para tareas de secuencias, como la generación de música, ya que pueden capturar dependencias a largo plazo en los datos.
Se utilizará la biblioteca `torch` para construir el modelo. A continuación se presenta un ejemplo de cómo se puede definir una clase para el modelo:

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
from tqdm import tqdm

def train_model(
        model: nn.Module, 
        dataloader: DataLoader, 
        epochs: int, 
        criterion_pitch: nn.Module, 
        criterion_reg: nn.Module, 
        optimizer: torch.optim.Optimizer, 
        device: torch.device, 
        alpha: float = 1.0,
        beta: float = 1.0,
        gamma: float = 1.0,
    ):
    """
    Entrena el modelo con pérdidas combinadas (pitch clasificación + regresión).
    Args:
        model (nn.Module): The LSTM model to train.
        dataloader (DataLoader): DataLoader for the training data.
        epochs (int): Number of epochs to train.
        criterion_pitch (nn.Module): Loss function for pitch.
        criterion_reg (nn.Module): Loss function for regression targets.
        optimizer (torch.optim.Optimizer): Optimizer for training.
        device (torch.device): Device to train on (CPU or GPU).
        alpha (float): Weight for pitch loss.
        beta (float): Weight for step loss.
        gamma (float): Weight for duration and velocity loss.
    """

    # mover el modelo a la GPU si está disponible
    model.to(device)

    # activamos el modo de entrenamiento
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        pitch_correct = 0
        pitch_total = 0

        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
        
        for batch_x, batch_y in loop:

            x_pitch, x_step, x_duration, x_velocity = batch_x
            y_pitch, y_step, y_duration, y_velocity = batch_y

            # mover los datos a la GPU si está disponible
            x_pitch = x_pitch.to(device)
            x_step = x_step.to(device)
            x_duration = x_duration.to(device)
            x_velocity = x_velocity.to(device)

            y_pitch = y_pitch.to(device)
            y_step = y_step.to(device).unsqueeze(1) # unsqueeze para que tenga la misma dimensión que el resto
            y_duration = y_duration.to(device).unsqueeze(1)
            y_velocity = y_velocity.to(device).unsqueeze(1)

            # Limpiar gradientes
            optimizer.zero_grad()

            # forward pass
            pitch_pred, step_pred, duration_pred, velocity_pred = model(x_pitch, x_step, x_duration, x_velocity)

            # calcular la pérdida para cada una de las características
            loss_pitch = criterion_pitch(pitch_pred, y_pitch)
            loss_step = criterion_reg(step_pred, y_step)
            loss_duration = criterion_reg(duration_pred, y_duration)
            loss_velocity = criterion_reg(velocity_pred, y_velocity)

            loss = loss_pitch + alpha * loss_step + beta * loss_duration + gamma * loss_velocity

            # backward pass y optimización
            loss.backward()
            optimizer.step()

            # Agregar la pérdida total
            total_loss += loss.item()


            # calcular la precisión de la predicción de pitch
            pitch_pred = torch.argmax(pitch_pred, dim=1)
            pitch_correct += (pitch_pred == y_pitch).sum().item()
            pitch_total += y_pitch.size(0)

        pitch_acc = pitch_correct / pitch_total * 100 if pitch_total > 0 else 0

        # ímprimir la pérdida total por cada epoch 
        print(f"🎵 Epoch {epoch+1}/{epochs} — 💥 Loss: {total_loss:.4f} — 🎯 Pitch Accuracy: {pitch_acc:.2f}%")


In [12]:

# Hiperparámetros
hidden_size = 128
num_layers = 2
epochs = 15
learning_rate = 0.001

# Inicializar modelo
model = MusicLSTM(
    pitch_vocab_size=len(label_encoder.classes_),
    embedding_dim=32,
    hidden_size=hidden_size,
    num_layers=num_layers
)

# Funciones de pérdida
criterion_pitch = nn.CrossEntropyLoss()
criterion_reg = nn.MSELoss()

# Optimizador
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

# Llamar al entrenamiento
train_model(
    model=model,
    dataloader=train_loader,
    criterion_pitch=criterion_pitch,
    criterion_reg=criterion_reg,
    optimizer=optimizer,
    epochs=epochs,
    device=device,
    alpha=0.5,
    beta=1.0,
    gamma=0.5,
)


🎵 Epoch 1/15 — 💥 Loss: 4925.6378 — 🎯 Pitch Accuracy: 21.09%


🎵 Epoch 2/15 — 💥 Loss: 4308.1224 — 🎯 Pitch Accuracy: 29.79%


🎵 Epoch 3/15 — 💥 Loss: 3931.3115 — 🎯 Pitch Accuracy: 35.33%


🎵 Epoch 4/15 — 💥 Loss: 3643.0464 — 🎯 Pitch Accuracy: 39.88%


🎵 Epoch 5/15 — 💥 Loss: 3403.5646 — 🎯 Pitch Accuracy: 44.00%


...

🎵 Epoch 11/15 — 💥 Loss: 2460.9926 — 🎯 Pitch Accuracy: 59.08%


🎵 Epoch 12/15 — 💥 Loss: 2345.8372 — 🎯 Pitch Accuracy: 60.90%


🎵 Epoch 13/15 — 💥 Loss: 2253.2366 — 🎯 Pitch Accuracy: 62.48%


🎵 Epoch 14/15 — 💥 Loss: 2177.2739 — 🎯 Pitch Accuracy: 63.46%


🎵 Epoch 15/15 — 💥 Loss: 2102.5327 — 🎯 Pitch Accuracy: 64.80%


In [ ]:
# guardar modelo
#torch.save(model.state_dict(), "music_lstm_weights.pth")
#torch.save(model.state_dict(), "music_lstm_weights_2.pth")

In [15]:
def evaluate_model(
    model: nn.Module,
    dataloader: DataLoader,
    criterion_pitch: nn.Module,
    criterion_reg: nn.Module,
    device: torch.device,
    alpha: float = 1.0,
    beta: float = 1.0,
    gamma: float = 1.0,
) -> Tuple[float, float]:
    """
    Evalúa el modelo y retorna el loss total y la precisión del pitch.
    """

    model.eval()
    total_loss = 0
    correct_pitch = 0
    total_pitch = 0

    with torch.no_grad():
        for batch_x, batch_y in dataloader:

            x_pitch, x_step, x_duration, x_velocity = batch_x
            y_pitch, y_step, y_duration, y_velocity = batch_y

            # Mover al dispositivo
            x_pitch = x_pitch.to(device)
            x_step = x_step.to(device)
            x_duration = x_duration.to(device)
            x_velocity = x_velocity.to(device)

            y_pitch = y_pitch.to(device)
            y_step = y_step.to(device).unsqueeze(1)
            y_duration = y_duration.to(device).unsqueeze(1)
            y_velocity = y_velocity.to(device).unsqueeze(1)

            # Forward pass
            pitch_pred, step_pred, duration_pred, velocity_pred = model(x_pitch, x_step, x_duration, x_velocity)

            # Losses
            loss_pitch = criterion_pitch(pitch_pred, y_pitch)
            loss_step = criterion_reg(step_pred, y_step)
            loss_duration = criterion_reg(duration_pred, y_duration)
            loss_velocity = criterion_reg(velocity_pred, y_velocity)

            loss = loss_pitch + alpha * loss_step + beta * loss_duration + gamma * loss_velocity
            total_loss += loss.item()

            # Calcular precisión del pitch
            pred_labels = torch.argmax(pitch_pred, dim=1)
            correct_pitch += (pred_labels == y_pitch).sum().item()
            total_pitch += y_pitch.size(0)

    avg_loss = total_loss / len(dataloader)
    pitch_accuracy = 100 * correct_pitch / total_pitch

    print(f"📊 Validation — Loss: {avg_loss:.4f} — 🎯 Pitch Accuracy: {pitch_accuracy:.2f}%")

    return avg_loss, pitch_accuracy


In [16]:
evaluate_model(
    model=model,
    dataloader=test_loader,
    criterion_pitch=criterion_pitch,
    criterion_reg=criterion_reg,
    device=device,
)

📊 Validation — Loss: 2.9114 — 🎯 Pitch Accuracy: 28.30%


(2.9114382858002994, 28.296122209165688)

In [18]:
def generate_sequence(model, seed_sequence, length, context=10, label_encoder=None, scaler=None, device='cpu'):
    """
    Genera una secuencia de notas a partir de una semilla.

    Args:
        model: modelo entrenado
        seed_sequence: lista de notas [[pitch_name, step, duration, velocity], ...]
        length: número de notas a generar
        context: tamaño de ventana de contexto
        label_encoder: LabelEncoder entrenado para los pitch
        scaler: StandardScaler entrenado para step, duration y velocity
        device: 'cpu' o 'cuda'
    
    Returns:
        Lista de notas generadas en formato [pitch_name, step, duration, velocity]
    """
    model.eval()
    generated = []

    current_seq = seed_sequence.copy()

    for _ in range(length):
        # Tomar las últimas `context` notas
        context_seq = current_seq[-context:]

        # Codificar pitch + normalizar numéricos
        encoded_seq = []
        for note in context_seq:
            pitch_encoded = label_encoder.transform([note[0]])[0]
            scaled = scaler.transform([[note[1], note[2], note[3]]])[0]
            encoded_seq.append([pitch_encoded, *scaled])

        # Crear input tensor con forma (1, context, 4)
        input_tensor = torch.tensor(encoded_seq, dtype=torch.float32).unsqueeze(0).to(device)

        # Separar por características
        x_pitch = input_tensor[:, :, 0].long().unsqueeze(-1)      # (1, context, 1)
        x_step = input_tensor[:, :, 1].unsqueeze(-1)              # (1, context, 1)
        x_duration = input_tensor[:, :, 2].unsqueeze(-1)          # (1, context, 1)
        x_velocity = input_tensor[:, :, 3].unsqueeze(-1)          # (1, context, 1)

        with torch.no_grad():
            pitch_logits, step, duration, velocity = model(x_pitch, x_step, x_duration, x_velocity)

        # Decodificar pitch
        pitch_idx = torch.argmax(pitch_logits, dim=1).item()
        pitch_name = label_encoder.inverse_transform([pitch_idx])[0]

        # Desnormalizar valores continuos
        step_val, duration_val, velocity_val = scaler.inverse_transform(
            [[step.item(), duration.item(), velocity.item()]]
        )[0]

        # ⚠️ Clipping para evitar valores absurdos
        step_val = np.clip(step_val, 0.01, 1.0)
        duration_val = np.clip(duration_val, 0.05, 3.0)
        velocity_val = np.clip(velocity_val, 0.1, 1.0)

        # Añadir nota generada
        generated.append([pitch_name, step_val, duration_val, velocity_val])
        current_seq.append([pitch_name, step_val, duration_val, velocity_val])

    return generated


In [ ]:
def get_seed_from_dataset(dataset, index, label_encoder):
    (x_pitch, x_step, x_duration, x_velocity), _ = dataset[index]
    seed = []
    for i in range(len(x_pitch)):
        pitch_name = label_encoder.inverse_transform([x_pitch[i, 0].item()])[0]
        step = x_step[i, 0].item()
        duration = x_duration[i, 0].item()
        velocity = x_velocity[i, 0].item()
        seed.append([pitch_name, step, duration, velocity])
    return seed

In [ ]:
# Parámetros
context = 32
num_notes = 200
# device = 'cpu'  # o 'cuda' si estás usando GPU
# label_encoder = train_dataset.label_encoder
scaler = preprocessor.scaler  # ya entrenado también

# Semilla: las primeras 10 notas del set de entrenamiento
seed_sequence = get_seed_from_dataset(train_dataset, 0, label_encoder=label_encoder)

# Generar notas
generated_notes = generate_sequence(
    model=model,
    seed_sequence=seed_sequence,
    length=num_notes,
    context=context,
    device=device,
    label_encoder=label_encoder,
    scaler=scaler,
)


In [25]:
generated_notes

[[np.int64(78),
  np.float64(0.14941015954403875),
  np.float64(0.17888646108016226),
  np.float64(1.0)],
 [np.int64(76),
  np.float64(0.11747538953250879),
  np.float64(0.24254695141903432),
  np.float64(1.0)],
 [np.int64(74),
  np.float64(0.16007862809121925),
  np.float64(0.2739891032845474),
  np.float64(1.0)],
 [np.int64(73),
  np.float64(0.1256848741572375),
  np.float64(0.20737775195608868),
  np.float64(1.0)],
 [np.int64(71),
  np.float64(0.11024226059836367),
  np.float64(0.16221757115031474),
  np.float64(1.0)],
 [np.int64(69),
  np.float64(0.2355564619081961),
  np.float64(0.1434262535709394),
  np.float64(1.0)],
 [np.int64(71),
  np.float64(0.19156289385661862),
  np.float64(0.11721698529602179),
  np.float64(1.0)],
 [np.int64(73),
  np.float64(0.1785147296290433),
  np.float64(0.14552893386672494),
  np.float64(1.0)],
 [np.int64(74),
  np.float64(0.2147927445724053),
  np.float64(0.10134500253964039),
  np.float64(1.0)],
 [np.int64(76),
  np.float64(0.22833770767844175),
 

In [22]:
def notes_to_midi(notes, output_file="output.mid"):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)

    start_time = 0
    for pitch, step, duration, velocity in notes:
        start_time += step
        end_time = start_time + duration
        note = pretty_midi.Note(velocity=int(velocity), pitch=pitch, start=start_time, end=end_time)
        instrument.notes.append(note)

    midi.instruments.append(instrument)
    midi.write(output_file)


In [26]:
notes_to_midi(
    notes=generated_notes,
    output_file='generated_music_28%_c16.mid',
)